In [64]:
import pandas as pd 
import numpy as np

import geopandas as gpd


import seaborn as sns

# Livrarias para ler dados do openstreetmaps e representação da rede de estradas como um grafo
#import osmnx as ox
import networkx as nx

import re

#import pgeocode as pgc
#import geopy

import time

import matplotlib.pyplot as plt

import os

import pickle 


from abc import abstractmethod
import torch




In [65]:
# from tqdm import tqdm
#{progress in percentage} | {bar} | {processed}/{total} [{elapsed time}<{remaining time}, {number of iterations per seconds} it/s]

In [66]:
os.getcwd()

'c:\\Users\\pauloricardolb\\OneDrive - Universidade de Aveiro\\TRABALHO_SCRIPS\\py_GeoIdentifZones\\Main\\PreProcessing'

# Directorias

In [67]:
#PC UA
pathUA = "C:/Users/pauloricardolb/"
pathHPCasa = "C:/Users/Paulo B/" 

path = pathUA



In [68]:
# SOURCE FOR PIKLE DATA
sourcePIKLE = path + "OneDrive - Universidade de Aveiro/TRABALHO_SCRIPS/py_GeoIdentifZones/DATA_Pikle"

# BASE FUNCTIONS

In [69]:
# Code retrieve from here: https://datascience.stackexchange.com/questions/45459/how-to-use-scikit-learn-label-propagation-on-graph-structured-data/55720#55720 
# »» label propagation/label spreading on graph-structured data
# »» https://scikit-learn.org/stable/modules/generated/sklearn.semi_supervised.LabelPropagation.html  

class BaseLabelPropagation:
    """Base class for label propagation models.
    
    Parameters
    ----------
    adj_matrix: torch.FloatTensor
        Adjacency matrix of the graph.
    """
    def __init__(self, adj_matrix):
        self.norm_adj_matrix = self._normalize(adj_matrix)
        self.n_nodes = adj_matrix.size(0)
        self.one_hot_labels = None 
        self.n_classes = None
        self.labeled_mask = None
        self.predictions = None

    @staticmethod
    @abstractmethod
    def _normalize(adj_matrix):
        raise NotImplementedError("_normalize must be implemented")

    @abstractmethod
    def _propagate(self):
        raise NotImplementedError("_propagate must be implemented")

    def _one_hot_encode(self, labels):
        # Get the number of classes
        classes = torch.unique(labels)
        classes = classes[classes != -1]
        self.n_classes = classes.size(0)

        # One-hot encode labeled data instances and zero rows corresponding to unlabeled instances
        unlabeled_mask = (labels == -1)
        labels = labels.clone()  # defensive copying
        labels[unlabeled_mask] = 0
        self.one_hot_labels = torch.zeros((self.n_nodes, self.n_classes), dtype=torch.float)
        self.one_hot_labels = self.one_hot_labels.scatter(1, labels.unsqueeze(1), 1)
        self.one_hot_labels[unlabeled_mask, 0] = 0

        self.labeled_mask = ~unlabeled_mask

    def fit(self, labels, max_iter, tol):
        """Fits a semi-supervised learning label propagation model.
        
        labels: torch.LongTensor
            Tensor of size n_nodes indicating the class number of each node.
            Unlabeled nodes are denoted with -1.
        max_iter: int
            Maximum number of iterations allowed.
        tol: float
            Convergence tolerance: threshold to consider the system at steady state.
        """
        self._one_hot_encode(labels)

        self.predictions = self.one_hot_labels.clone()
        prev_predictions = torch.zeros((self.n_nodes, self.n_classes), dtype=torch.float)

        for i in range(max_iter):
            # Stop iterations if the system is considered at a steady state
            variation = torch.abs(self.predictions - prev_predictions).sum().item()
            
            if variation < tol:
                print(f"The method stopped after {i} iterations, variation={variation:.4f}.")
                break

            prev_predictions = self.predictions
            self._propagate()

    def predict(self):
        return self.predictions

    def predict_classes(self):
        return self.predictions.max(dim=1).indices

# LABEL PROPAGATION ALGORITHM

class LabelPropagation(BaseLabelPropagation):
    def __init__(self, adj_matrix):
        super().__init__(adj_matrix)

    @staticmethod
    def _normalize(adj_matrix):
        """Computes D^-1 * W"""
        degs = adj_matrix.sum(dim=1)
        degs[degs == 0] = 1  # avoid division by 0 error
        return adj_matrix / degs[:, None]

    def _propagate(self):
        self.predictions = torch.matmul(self.norm_adj_matrix, self.predictions)

        # Put back already known labels
        self.predictions[self.labeled_mask] = self.one_hot_labels[self.labeled_mask]

    def fit(self, labels, max_iter=1000, tol=1e-3):
        super().fit(labels, max_iter, tol)


class LabelSpreading(BaseLabelPropagation):
    def __init__(self, adj_matrix):
        super().__init__(adj_matrix)
        self.alpha = None

    @staticmethod
    def _normalize(adj_matrix):
        """Computes D^-1/2 * W * D^-1/2"""
        degs = adj_matrix.sum(dim=1)
        norm = torch.pow(degs, -0.5)
        norm[torch.isinf(norm)] = 1
        return adj_matrix * norm[:, None] * norm[None, :]

    def _propagate(self):
        self.predictions = (
            self.alpha * torch.matmul(self.norm_adj_matrix, self.predictions)
            + (1 - self.alpha) * self.one_hot_labels
        )
    
    def fit(self, labels, max_iter=1000, tol=1e-3, alpha=0.5):
        """
        Parameters
        ----------
        alpha: float
            Clamping factor.
        """
        self.alpha = alpha
        super().fit(labels, max_iter, tol)


# MODEL 1 :: AVRILH ORI NET

## Get Data

In [70]:
avrilh_network_wgs84 = pickle.load(open(sourcePIKLE + '/avrilh_network_wgs84.pkl', "rb"))
avrilh_network_wgs84_dfnodes = pickle.load(open(sourcePIKLE + '/avrilh_network_wgs84_dfnodes.pkl', "rb"))



C:\Users\pauloricardolb\AppData\Local\Temp\ipykernel_30024\2415597971.py:1: UserWarning: Unpickling a shapely <2.0 geometry object. Please save the pickle again; shapely 2.1 will not have this compatibility.
  avrilh_network_wgs84 = pickle.load(open(sourcePIKLE + '/avrilh_network_wgs84.pkl', "rb"))


## Pre Processing

### Build Adjacency Matrix - Contiguity

In [71]:
Adj_matrix_contig = nx.adjacency_matrix(avrilh_network_wgs84).toarray() #weight='length'
Adj_matrix_contig

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0]], dtype=int32)

In [72]:
Adj_matrix_contig.shape

(10886, 10886)

### Build Adjacency Matrix - GeoNetDistance

In [73]:
Adj_matrix_dist = nx.adjacency_matrix(avrilh_network_wgs84, weight='length').toarray() #weight='length'
Adj_matrix_dist

array([[  0.   , 663.071,   0.   , ...,   0.   ,   0.   ,   0.   ],
       [  0.   ,   0.   ,   0.   , ...,   0.   ,   0.   ,   0.   ],
       [  0.   ,   0.   ,   0.   , ...,   0.   ,   0.   ,   0.   ],
       ...,
       [  0.   ,   0.   ,   0.   , ...,   0.   ,   0.   ,   0.   ],
       [  0.   ,   0.   ,   0.   , ...,   0.   ,   0.   , 655.549],
       [  0.   ,   0.   ,   0.   , ...,   0.   , 655.549,   0.   ]])

In [74]:
#Normalize adjacency distance matrix by row

import sklearn.preprocessing
Adj_matrix_dist_normal = sklearn.preprocessing.normalize(Adj_matrix_dist, norm="l1")
print(Adj_matrix_dist_normal.shape)
Adj_matrix_dist_normal


(10886, 10886)


array([[0.        , 0.6232708 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.86724761],
       [0.        , 0.        , 0.        , ..., 0.        , 0.43360057,
        0.        ]])

In [75]:
avrilh_network_wgs84_dfnodes.columns

Index(['y', 'x', 'ref', 'highway', 'street_count', 'NODE_ZONE_CS'], dtype='object')

In [76]:
avrilh_network_wgs84_dfnodes[["NODE_ZONE_CS"]]

,NODE_ZONE_CS
26018652,-1
26018669,-1
26018671,-1
26019628,-1
26019631,-1
...,...
10691051533,-1
10691051542,-1
10691051583,-1
10691051596,-1


In [77]:
avrilh_network_wgs84_dfnodes[["NODE_ZONE_CS"]].values.flatten()


array([-1, -1, -1, ..., -1, -1, -1], dtype=int64)

### Build PyTorch objects

In [78]:
import torch


labels_t = torch.LongTensor(avrilh_network_wgs84_dfnodes[["NODE_ZONE_CS"]].values.flatten())

# Create input tensors
Adj_matrix_dist_RowNorm_t = torch.FloatTensor(Adj_matrix_dist_normal)
Adj_matrix_contig_t = torch.FloatTensor(Adj_matrix_contig)





In [79]:
labels_t.shape

torch.Size([10886])

In [80]:
Adj_matrix_dist_RowNorm_t.shape

torch.Size([10886, 10886])

## Modelling

### Modelling 1a1 :: LabelPropag + AdjMat_Contiguity

In [81]:
# Learn with Label Propagation
model1a1_LP_AdjMatCont_AVRILH_Ori_FullNetwork = LabelPropagation(Adj_matrix_contig_t)
print("Model 1 - Label Propagatiom :: AdjMatrix Contig : ", end="")
model1a1_LP_AdjMatCont_AVRILH_Ori_FullNetwork.fit(labels_t)
model1a1_LP_AdjMatCont_AVRILH_Ori_FullNetwork_LABELS = model1a1_LP_AdjMatCont_AVRILH_Ori_FullNetwork.predict_classes()
model1a1_LP_AdjMatCont_AVRILH_Ori_FullNetwork_LABELS

Model 1 - Label Propagatiom :: AdjMatrix Contig : 

tensor([72,  0, 72,  ..., 98, 98, 98])

### Modelling 1a2 :: LabelPropag + AdjMat_GeoNetDistance

In [82]:
# Learn with Label Propagation
model1a2_LP_AdjMatCont_AVRILH_Ori_FullNetwork = LabelPropagation(Adj_matrix_dist_RowNorm_t)
print("Model 1 - Label Propagation :: AdjMatrix GeoNetDist : ", end="")
model1a2_LP_AdjMatCont_AVRILH_Ori_FullNetwork.fit(labels_t)
model1a2_LP_AdjMatCont_AVRILH_Ori_FullNetwork_LABELS = model1a2_LP_AdjMatCont_AVRILH_Ori_FullNetwork.predict_classes()
model1a2_LP_AdjMatCont_AVRILH_Ori_FullNetwork_LABELS

Model 1 - Label Propagation :: AdjMatrix GeoNetDist : 

tensor([72,  0, 72,  ..., 98, 98, 98])

### Modelling 1b1 :: LabelSpread + AdjMat_Contiguity

In [83]:
# Learn with Label Propagation
model1b1_LP_AdjMatCont_AVRILH_Ori_FullNetwork = LabelSpreading(Adj_matrix_contig_t)
print("Model 1 - Label Spreading :: AdjMatrix Contig : ", end="")
model1b1_LP_AdjMatCont_AVRILH_Ori_FullNetwork.fit(labels_t)
model1b1_LP_AdjMatCont_AVRILH_Ori_FullNetwork_LABELS = model1b1_LP_AdjMatCont_AVRILH_Ori_FullNetwork.predict_classes()
model1b1_LP_AdjMatCont_AVRILH_Ori_FullNetwork_LABELS

Model 1 - Label Spreading :: AdjMatrix Contig : The method stopped after 17 iterations, variation=0.0007.


tensor([72,  0, 72,  ..., 98, 98, 98])

### Modelling 1b2 :: LabelSpread + AdjMat_GeoNetDistance

In [84]:
# Learn with Label Propagation
model1b2_LP_AdjMatCont_AVRILH_Ori_FullNetwork = LabelSpreading(Adj_matrix_dist_RowNorm_t)
print("Model 1 - Label Spreading :: AdjMatrix GeoNetDist : ", end="")
model1b2_LP_AdjMatCont_AVRILH_Ori_FullNetwork.fit(labels_t)
model1b2_LP_AdjMatCont_AVRILH_Ori_FullNetwork_LABELS = model1b2_LP_AdjMatCont_AVRILH_Ori_FullNetwork.predict_classes()
model1b2_LP_AdjMatCont_AVRILH_Ori_FullNetwork_LABELS

Model 1 - Label Spreading :: AdjMatrix GeoNetDist : The method stopped after 18 iterations, variation=0.0007.


tensor([72,  0, 72,  ..., 98, 98, 98])

## Save results to nodes geodataframe

In [85]:
avrilh_network_wgs84_dfnodes.head()

,y,x,ref,highway,street_count,NODE_ZONE_CS
26018652,40.573359,-8.561106,15,motorway_junction,3,-1
26018669,40.567454,-8.560672,NaN,NaN,3,-1
26018671,40.568421,-8.560302,15,motorway_junction,3,-1
26019628,40.627240,-8.730450,1,motorway_junction,3,-1
26019631,40.625874,-8.726227,1A,motorway_junction,3,-1


In [86]:
avrilh_network_wgs84_dfnodes_LABELED_Part1 = avrilh_network_wgs84_dfnodes.copy()

In [87]:
avrilh_network_wgs84_dfnodes_LABELED_Part1.reset_index(inplace=True)
avrilh_network_wgs84_dfnodes_LABELED_Part1.drop(columns=["ref",	"highway", "street_count"], inplace=True)
avrilh_network_wgs84_dfnodes_LABELED_Part1.rename(columns={"index":"NODE_ID"}, inplace=True)
print(type(avrilh_network_wgs84_dfnodes_LABELED_Part1))
avrilh_network_wgs84_dfnodes_LABELED_Part1.head()

<class 'pandas.core.frame.DataFrame'>


,NODE_ID,y,x,NODE_ZONE_CS
0,26018652,40.573359,-8.561106,-1
1,26018669,40.567454,-8.560672,-1
2,26018671,40.568421,-8.560302,-1
3,26019628,40.627240,-8.730450,-1
4,26019631,40.625874,-8.726227,-1


In [88]:
avrilh_network_wgs84_dfnodes_LABELED_Part1["M1LABELS_LP_AdjMatContig"] = model1a1_LP_AdjMatCont_AVRILH_Ori_FullNetwork_LABELS
avrilh_network_wgs84_dfnodes_LABELED_Part1["M1LABELS_LP_AdjMatGeoNetDist"] = model1a2_LP_AdjMatCont_AVRILH_Ori_FullNetwork_LABELS
avrilh_network_wgs84_dfnodes_LABELED_Part1["M1LABELS_LS_AdjMatContig"] = model1b1_LP_AdjMatCont_AVRILH_Ori_FullNetwork_LABELS
avrilh_network_wgs84_dfnodes_LABELED_Part1["M1LABELS_LS_AdjMatGeoNetDist"] = model1b2_LP_AdjMatCont_AVRILH_Ori_FullNetwork_LABELS
avrilh_network_wgs84_dfnodes_LABELED_Part1.head()

,NODE_ID,y,x,NODE_ZONE_CS,M1LABELS_LP_AdjMatContig,M1LABELS_LP_AdjMatGeoNetDist,M1LABELS_LS_AdjMatContig,M1LABELS_LS_AdjMatGeoNetDist
0,26018652,40.573359,-8.561106,-1,72,72,72,72
1,26018669,40.567454,-8.560672,-1,0,0,0,0
2,26018671,40.568421,-8.560302,-1,72,72,72,72
3,26019628,40.627240,-8.730450,-1,81,81,49,49
4,26019631,40.625874,-8.726227,-1,50,86,50,86


## Save results to nodes geodataframe

In [89]:
avrilh_network_wgs84_dfnodes_LABELED_Part1.to_csv("avrilh_network_wgs84_dfnodes_LABELED_Part1.csv", index=False)

In [90]:
pd.pivot_table( avrilh_network_wgs84_dfnodes_LABELED_Part1 , values="NODE_ZONE_CS", index="M1LABELS_LP_AdjMatGeoNetDist", columns=None, aggfunc=np.count_nonzero, fill_value=0, margins=False, dropna=True, margins_name='All', observed=False, sort=True)

,NODE_ZONE_CS
M1LABELS_LP_AdjMatGeoNetDist,
0,45
1,10
2,183
3,96
4,117
...,...
127,55
128,54
129,21


In [91]:
avrilh_network_wgs84_dfnodes_LABELED_Part1.head()

,NODE_ID,y,x,NODE_ZONE_CS,M1LABELS_LP_AdjMatContig,M1LABELS_LP_AdjMatGeoNetDist,M1LABELS_LS_AdjMatContig,M1LABELS_LS_AdjMatGeoNetDist
0,26018652,40.573359,-8.561106,-1,72,72,72,72
1,26018669,40.567454,-8.560672,-1,0,0,0,0
2,26018671,40.568421,-8.560302,-1,72,72,72,72
3,26019628,40.627240,-8.730450,-1,81,81,49,49
4,26019631,40.625874,-8.726227,-1,50,86,50,86


In [92]:
open_file = open(sourcePIKLE + '/avrilh_network_wgs84_dfnodes_LABELED_Part1.pkl', "wb")
pickle.dump(avrilh_network_wgs84_dfnodes_LABELED_Part1, open_file)
open_file.close()

# MODEL 2 :: AVRILH SIMPLIFIED NET

## Get Data

In [77]:
AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild = pickle.load(open(sourcePIKLE + '/AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild.pkl', "rb"))
AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild_nodesF = pickle.load(open(sourcePIKLE + '/AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild_nodesF.pkl', "rb"))



C:\Users\pauloricardolb\AppData\Local\Temp\ipykernel_3512\2662819000.py:1: UserWarning: Unpickling a shapely <2.0 geometry object. Please save the pickle again; shapely 2.1 will not have this compatibility.
  AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild = pickle.load(open(sourcePIKLE + '/AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild.pkl', "rb"))
C:\Users\pauloricardolb\AppData\Local\Temp\ipykernel_3512\2662819000.py:2: UserWarning: Unpickling a shapely <2.0 geometry object. Please save the pickle again; shapely 2.1 will not have this compatibility.
  AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild_nodesF = pickle.load(open(sourcePIKLE + '/AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild_nodesF.pkl', "rb"))


In [78]:
print(type(AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild_nodesF))
AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild_nodesF.head()

<class 'geopandas.geodataframe.GeoDataFrame'>


,geometry,zone_id
26018652,POINT (-8.56111 40.57336),-1
503706150,POINT (-8.56288 40.57063),-1
26018669,POINT (-8.56067 40.56745),-1
26018671,POINT (-8.56030 40.56842),-1
1713444500,POINT (-8.56044 40.57157),-1


## Pre Processing

### Build Adjacency Matrix - Contiguity

In [79]:
AdjMatrix_Contig = nx.adjacency_matrix(AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild).toarray() #weight='length'
AdjMatrix_Contig

array([[0., 1., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [80]:
AdjMatrix_Contig.shape

(7301, 7301)

### Build Adjacency Matrix - GeoNetDistance

In [40]:
AdjMatrix_GeoNetDist = nx.adjacency_matrix(AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild, weight='length').toarray() #weight='length'
AdjMatrix_GeoNetDist

array([[  0.   , 400.786, 663.071, ...,   0.   ,   0.   ,   0.   ],
       [400.786,   0.   ,   0.   , ...,   0.   ,   0.   ,   0.   ],
       [663.071,   0.   ,   0.   , ...,   0.   ,   0.   ,   0.   ],
       ...,
       [  0.   ,   0.   ,   0.   , ...,   0.   ,   0.   ,   0.   ],
       [  0.   ,   0.   ,   0.   , ...,   0.   ,   0.   ,   0.   ],
       [  0.   ,   0.   ,   0.   , ...,   0.   ,   0.   ,   0.   ]])

In [41]:
#Normalize adjacency distance matrix by row

import sklearn.preprocessing
AdjMatrix_GeoNetDist_RowNorm = sklearn.preprocessing.normalize(AdjMatrix_GeoNetDist, norm="l1")
print(AdjMatrix_GeoNetDist_RowNorm.shape)
AdjMatrix_GeoNetDist_RowNorm


(7301, 7301)


array([[0.        , 0.3767292 , 0.6232708 , ..., 0.        , 0.        ,
        0.        ],
       [0.25478987, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.57923638, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [42]:
AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild_nodesF.columns

Index(['geometry', 'zone_id'], dtype='object')

In [43]:
AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild_nodesF[["zone_id"]]

,zone_id
26018652,-1
503706150,-1
26018669,-1
26018671,-1
1713444500,-1
...,...
10570422468,-1
10691051301,-1
10691051379,-1
10691051396,-1


In [44]:
np.unique(AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild_nodesF[["zone_id"]].values)

array([ -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
        12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,
        25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,
        38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
        51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,
        64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
        77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
       103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
       116, 117, 118, 119, 120, 121, 122], dtype=int64)

### Build PyTorch objects

In [45]:
import torch


labels_t = torch.LongTensor(AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild_nodesF[["zone_id"]].values.flatten())

# Create input tensors
AdjMatrix_Contig_t = torch.FloatTensor(AdjMatrix_Contig)
AdjMatrix_GeoNetDist_RowNorm_t = torch.FloatTensor(AdjMatrix_GeoNetDist_RowNorm)





In [46]:
labels_t.shape

torch.Size([7301])

## Modelling

### Modelling 2a1 :: LabelPropag + AdjMat_Contiguity

In [50]:
# Learn with Label Propagation
model2a1_LP_AdjMatCont_AVRILH_network_simpleGraph = LabelPropagation(AdjMatrix_Contig_t)
print("Label Propagatiom :: AdjMatrix Contig : ", end="")
model2a1_LP_AdjMatCont_AVRILH_network_simpleGraph.fit(labels_t)
model2a1_LP_AdjMatCont_AVRILH_network_simpleGraph_LABELS = model2a1_LP_AdjMatCont_AVRILH_network_simpleGraph.predict_classes()
model2a1_LP_AdjMatCont_AVRILH_network_simpleGraph_LABELS

Label Propagatiom :: AdjMatrix Contig : 

tensor([32, 32, 32,  ..., 90, 90, 93])

### Modelling 2a2 :: LabelPropag + AdjMat_GeoNetDistance

In [51]:
# Learn with Label Propagation
model2a2_LP_AdjMatGeoNetDist_AVRILH_network_simpleGraph = LabelPropagation(AdjMatrix_GeoNetDist_RowNorm_t)
print("Label Propagation :: AdjMatrix GeoNetDist : ", end="")
model2a2_LP_AdjMatGeoNetDist_AVRILH_network_simpleGraph.fit(labels_t)
model2a2_LP_AdjMatGeoNetDist_AVRILH_network_simpleGraph_LABELS = model2a2_LP_AdjMatGeoNetDist_AVRILH_network_simpleGraph.predict_classes()
model2a2_LP_AdjMatGeoNetDist_AVRILH_network_simpleGraph_LABELS

Label Propagation :: AdjMatrix GeoNetDist : 

tensor([32, 32, 32,  ..., 90, 90, 93])

### Modelling 2b1 :: LabelSpread + AdjMat_Contiguity

In [56]:
# Learn with Label Propagation
model2b1_LS_AdjMatCont_AVRILH_network_simpleGraph = LabelSpreading(AdjMatrix_Contig_t)
print("Label Spreading :: AdjMatrix Contig : ", end="")
model2b1_LS_AdjMatCont_AVRILH_network_simpleGraph.fit(labels_t)
model2b1_LS_AdjMatCont_AVRILH_network_simpleGraph_LABELS = model2b1_LS_AdjMatCont_AVRILH_network_simpleGraph.predict_classes()
model2b1_LS_AdjMatCont_AVRILH_network_simpleGraph_LABELS

Label Spreading :: AdjMatrix Contig : The method stopped after 15 iterations, variation=0.0010.


tensor([32, 32, 32,  ..., 12, 12, 93])

### Modelling 2b2 :: LabelSpread + AdjMat_GeoNetDistance

In [57]:
# Learn with Label Propagation
model2b2_LS_AdjMatGeoNetDist_AVRILH_network_simpleGraph = LabelSpreading(AdjMatrix_GeoNetDist_RowNorm_t)
print("Label Spreading :: AdjMatrix GeoNetDist : ", end="")
model2b2_LS_AdjMatGeoNetDist_AVRILH_network_simpleGraph.fit(labels_t)
model2b2_LS_AdjMatGeoNetDist_AVRILH_network_simpleGraph_LABELS = model2b2_LS_AdjMatGeoNetDist_AVRILH_network_simpleGraph.predict_classes()
model2b2_LS_AdjMatGeoNetDist_AVRILH_network_simpleGraph_LABELS

Label Spreading :: AdjMatrix GeoNetDist : The method stopped after 17 iterations, variation=0.0008.


tensor([32, 32, 32,  ..., 90, 90, 93])

## Save results to nodes geodataframe

In [81]:
AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild_nodesF.reset_index(inplace=True)
AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild_nodesF.rename(columns={"index":"NODE_ID"}, inplace=True)
print(type(AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild_nodesF))
AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild_nodesF.head()

<class 'geopandas.geodataframe.GeoDataFrame'>


,NODE_ID,geometry,zone_id
0,26018652,POINT (-8.56111 40.57336),-1
1,503706150,POINT (-8.56288 40.57063),-1
2,26018669,POINT (-8.56067 40.56745),-1
3,26018671,POINT (-8.56030 40.56842),-1
4,1713444500,POINT (-8.56044 40.57157),-1


In [82]:
AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild_nodesF["LABELS_LP_AdjMatContig"] = model2a1_LP_AdjMatCont_AVRILH_network_simpleGraph_LABELS
AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild_nodesF["LABELS_LP_AdjMatGeoNetDist"] = model2a2_LP_AdjMatGeoNetDist_AVRILH_network_simpleGraph_LABELS
AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild_nodesF["LABELS_LS_AdjMatContig"] = model2b1_LS_AdjMatCont_AVRILH_network_simpleGraph_LABELS
AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild_nodesF["LABELS_LS_AdjMatGeoNetDist"] = model2b2_LS_AdjMatGeoNetDist_AVRILH_network_simpleGraph_LABELS
AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild_nodesF.head()

,NODE_ID,geometry,zone_id,LABELS_LP_AdjMatContig,LABELS_LP_AdjMatGeoNetDist,LABELS_LS_AdjMatContig,LABELS_LS_AdjMatGeoNetDist
0,26018652,POINT (-8.56111 40.57336),-1,32,32,32,32
1,503706150,POINT (-8.56288 40.57063),-1,32,32,32,32
2,26018669,POINT (-8.56067 40.56745),-1,32,32,32,32
3,26018671,POINT (-8.56030 40.56842),-1,32,32,32,32
4,1713444500,POINT (-8.56044 40.57157),-1,32,32,32,32


In [66]:
pd.pivot_table( AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild_nodesF , values=["NODE_ID"], index="LABELS_LS_AdjMatGeoNetDist", columns=None, aggfunc=np.count_nonzero, fill_value=0, margins=False, dropna=True, margins_name='All', observed=False, sort=True)

,NODE_ID
LABELS_LS_AdjMatGeoNetDist,
0,58
1,100
2,1
3,64
4,2
...,...
118,38
119,13
120,1


In [68]:
open_file = open(sourcePIKLE + '/AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild_nodesF_labeled.pkl', "wb")
pickle.dump(AVRILH_network_simpleGraph_with_geom_aditInfo_rebuild_nodesF, open_file)
open_file.close()

In [69]:
sourcePIKLE

'C:/Users/pauloricardolb/OneDrive - Universidade de Aveiro/TRABALHO_SCRIPS/py_GeoIdentifZones/DATA_Pikle'